In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes

import torch
import psutil
import builtins

# 1. Global Fixes
builtins.psutil = psutil # Fixes the NameError

import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# 2. Configuration
max_seq_length = 2048
dtype = None          # Auto-detects best precision for your GPU
load_in_4bit = True   # <--- The Magic Switch (QLoRA)

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# 3. Load Model in 4-bit
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# 4. Attach LoRA Adapters
# This allows us to train only 1% of the params while keeping the rest frozen
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank (power of the adapter)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # optimized for long context
    random_state = 3407,
)

# 5. Data Preparation (With the fix for empty lines)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split = "train")
dataset = dataset.filter(lambda x: len(x["text"].strip()) > 0)

# 6. Trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # Low number prevents psutil crashes
    packing = False,      # False is safer for raw text start

    args = TrainingArguments(
        per_device_train_batch_size = 4, # We can increase this now!
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Remove this to train fully

        # Unsloth Optimized Settings
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit", # Saves even more memory
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "lora_outputs",
        report_to = "none",
    ),
)

# 7. Train
print("Starting QLoRA training...")
trainer_stats = trainer.train()

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-hsja7gm4/unsloth_174b623caca24a0c85e493e579ad3f00
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-hsja7gm4/unsloth_174b623caca24a0c85e493e579ad3f00
  Resolved https://github.com/unslothai/unsloth.git to commit d8459b194c81a502962f16118509cdfa205483af
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 15.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 

model.safetensors:   0%|          | 0.00/1.81G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

The model is already on multiple devices. Skipping the move to device specified in `args`.


Starting QLoRA training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 23,767 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 18,464,768 of 1,795,552,768 (1.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,4.600600
2,4.518500
3,4.225600
4,4.625600
5,4.211500
6,4.121100
7,4.254600
8,4.529300
9,3.641500
10,3.702800
